In [ ]:
import sys
import os
import json

# Cari path ke project_root dari lokasi notebook
notebook_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Tambahkan project root ke sys.path jika belum ada
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from utils.database import Database

db = Database()

db_metadata = db.impl.get_load_status()
db_metadata

In [ ]:
import os
from utils.database import Database
from script.ingestion import Ingestion

db = Database("postgresql", "localhost", 5432, "postgres", "postgres", "dvdrental")
ingestion = Ingestion(
                db, 
                "MINIO_ENDPOINT", 
                "MINIO_ACCESS_KEY", 
                "MINIO_SECRET_KEY", 
                "MINIO_SECURE", 
                "MINIO_BUCKET_NAME"
            )

ingestion.extract()

In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa 
import io
from minio import Minio
from datetime import datetime


client = Minio(
        "localhost:9000",
        access_key="admin_minio",
        secret_key="admin_minio",
        secure=False
    )



for schema, table_list in db_metadata.items():
    for table in table_list:
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        
        df = pd.read_sql(f"SELECT * FROM {schema}.{table['table']}", db.impl.engine)
        
        # pq.write_table(pa.Table.from_pandas(df), f"{table['table']}.parquet", compression="snappy")
        
        buffer = io.BytesIO()
        pq.write_table(pa.Table.from_pandas(df), buffer, compression="snappy")
        buffer.seek(0)
        
        client.put_object(
            bucket_name="landing-zones",
            object_name=f"{DB_NAME}/{schema}/{table['table']}/latest/{table['table']}.parquet",
            data=buffer,
            length=len(buffer.getvalue()),
            content_type="application/vnd.apache.parquet"
        )

In [ ]:
print(buffer.getvalue()[:4]) 

In [ ]:
# GET DATA LOAD ALL

import pandas as pd

df_chunks = pd.read_sql("SELECT * FROM category", db.impl.engine, chunksize=10000)
type(df_chunks)

for df in df_chunks:
    # print(df)
    break

dff = pd.read_sql("SELECT * FROM category", db.impl.engine)
df_list = []
df_list.append(dff)
for df in df_list:
    dfff = df
    
dfff

In [ ]:
# GET DATA LOAD INCREMENTAL

import pandas as pd 
from datetime import datetime

current_date = datetime.now().strftime("%Y-%m-%d")

df = pd.read_sql(f"SELECT * FROM inventory where updated_at >= '{current_date}'", db.impl.engine)
print(df.dtypes)

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd 
import io


df = pd.read_sql(f"SELECT * FROM film", db.impl.engine)
# print(df.dtypes)

pq.write_table(pa.Table.from_pandas(df), "film.parquet", compression="snappy")

# buffer = io.BytesIO()
# pq.write_table(pa.Table.from_pandas(df), buffer, compression="snappy")
# buffer.seek(0)

In [ ]:
from minio import Minio
from minio.error import S3Error

client = Minio(
        "localhost:9000",
        access_key="admin_minio",
        secret_key="admin_minio",
        secure=False
    )

client.put_object(
        bucket_name="data",
        object_name="public/staff.parquet",
        data=buffer,
        length=len(buffer.getvalue()),
        content_type="application/vnd.apache.parquet"
    )

In [ ]:
from minio import Minio
from minio.error import S3Error
from minio.commonconfig import CopySource


client = Minio(
        "localhost:9000",
        access_key="admin_minio",
        secret_key="admin_minio",
        secure=False
    )

# get bucket lists
buckets = client.list_buckets()
bucket_list = []
for bucket in buckets:
    bucket_list.append(bucket.name)

bucket_list
# ['landing-zones']

# list object
object_list = []
for object in client.list_objects(bucket_name="landing-zones", prefix="dvdrental/public/actor/latests/"):
    object_list.append(object.object_name)

if not object_list:
    print("object not found")

In [ ]:
import os

xx = os.getenv("TEST_ENV")
xx

In [ ]:
xx = ("a", "b")
if type(xx) == tuple:
    print("tuple")
else:
    print("not tuple")

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.schema import Table, MetaData
from sqlalchemy.sql.expression import select, text
import pandas as pd

engine = create_engine('trino://admin@localhost:8443/minio', connect_args={'http_scheme': 'https', 'verify': False})

df = pd.read_sql("SELECT * FROM minio.dvdrental.customer", engine)
df